In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath('../..'))


from offshore_wind_nj.data_loader import data_files, load_data, all_arrays, extract_datetime_from_filename, removed_files, load_single_data
from offshore_wind_nj.sar_wind_plot import plot_wind_field, plot_wind_field_by_arrays
from offshore_wind_nj.data_cleaning import fill_zeros, find_zeros
import offshore_wind_nj.data_processing as data_processing

2024-11-07 22:34:10.889 | INFO     | offshore_wind_nj.config:<module>:11 - PROJ_ROOT path is: C:\Users\crist\OneDrive - montclair.edu\00.MSU_MsDataScience\2024.FALL(Offshore_research)\03.CODING\00.CCNM_OffshoreWind2425(Github)


There are 11 files


# Get the filtered flattened data (No Nan values), file names & dates

In [2]:
flattened_data_list, filename_datetimes = data_processing.flatten_data(all_arrays, data_files, mask=True)

# Scale the data 
## if there are any error indices, then remove them from the filename_datetimes list

In [3]:
scaler = data_processing.scaler_flattened_data(flattened_data_list)
scaled_data_list, error_indices = data_processing.scale_flat_data(flattened_data_list, scaler)

if error_indices:
    for index in sorted(error_indices, reverse=True):
        del all_arrays[index]
        del data_files[index]
    print(f"The error indices were removed from the all_arrays ({len(all_arrays)}) & datafiles ({len(data_files)})")
else:
    print("No error indices")

No error indices


In [6]:
import pandas as pd
# Initialize a list to store each array's DataFrame with metadata
dataframes = []

# Process each array and corresponding metadata
for array, (ID, date, start_time, end_time) in zip(flattened_data_list, filename_datetimes):
    # Convert the array to a DataFrame
    array_df = pd.DataFrame(array, columns=['Speed','Direction', 'Cos(dir)', 'Sen(dir)', 'Lat', 'Lon'])
    
    # Add metadata columns to each row of the array DataFrame
    array_df['ID'] = ID
    array_df['Date'] = date
    array_df['Start Time'] = start_time
    array_df['End Time'] = end_time
    
    # Append to the list
    dataframes.append(array_df)

# Concatenate all dataframes
df = pd.concat(dataframes, ignore_index=True)

In [12]:
df.describe()

,Speed,Direction,Cos(dir),Sen(dir),Lat,Lon
count,469376.000000,469376.000000,469376.000000,469376.000000,469376.000000,469376.000000
mean,9.196745,194.110565,-0.245499,-0.080610,40.308575,-70.539680
std,3.284831,94.987091,0.821271,0.508605,1.639136,2.449620
min,0.003102,0.000000,-1.000000,-0.999998,35.735710,-75.330620
25%,7.105077,147.502384,-0.939774,-0.452039,39.627407,-72.506784
50%,8.899047,193.185150,-0.769499,-0.144915,40.706781,-70.887772
75%,11.689569,234.302559,0.715308,0.341952,41.421494,-68.396656
max,28.343872,359.999878,1.000000,0.999999,43.123699,-65.590340
